In [1]:
import pandas as pd
import pickle
import locale 

locale.setlocale(locale.LC_TIME, 'es_ES.utf8')

'es_ES.utf8'

In [3]:
categorias_de_aplicacion = pd.read_excel("relacion_insumos_principales_categorias.xlsx")

In [5]:
categorias_de_aplicacion.dropna(subset=["categorias_por_insumo"])

,insumo,descripcion_insumo,categorias_por_insumo
0,AA0045,"GLIBOR CALCIO (16,8% CaO+4,2% B)",Fertilizante
4,AA0016,10-30-10,Fertilizante
5,AA0058,10-30-10 + MENORES,Fertilizante
9,AC0284,ACIDO BORICO POLVO,Fertilizante
18,AI0064,ALISIN,insecticida
...,...,...,...
164,1016,SULFATO DE MAGNESIO,Fertilizante
170,AF0101,TRIVOR 310 DC,insecticida
173,AA0064,"VINAZA BIORGANIK (Humedad 1,20% Cenizas 25,8% ...",Fertilizante
175,AF0098,"ZOOM DIMETOMORPH, PROPAMUCARB",fungicida


In [29]:
categorias_de_aplicacion.iat[0,-1]

'herbicida'

In [9]:
categorias_de_aplicacion.iat[10,-1].split("'")

['{', 'herbicida', '}']

In [36]:
tooltip = [
        {
            column: {'value': str(value), 'type': 'markdown'}
            for column, value in row.items()
        } for row in nutricion_preforza_pc[["formula"]].to_dict('rows')
    ]

In [42]:
tooltip[3]

{'formula': {'value': 'FF-01', 'type': 'markdown'}}

In [75]:
insumos = pd.read_excel("insumos_por_aplicacion.xlsx")
insumos["agrupacion"] = insumos.groupby("id_formula")["Descripcion"].transform(lambda x: "\n".join(x))
insumos.drop_duplicates(subset=["id_formula"],inplace=True)
insumos["tooltip"]="Descripción aplicación: " + insumos.descripcion_formula + "\n"+insumos.agrupacion
diccionario_insumos = insumos.set_index("id_formula")["tooltip"].to_dict()

In [2]:
calendario_plan = pd.read_excel("calendario_aplicaciones.xlsx")
calendario_plan.drop(["y"],axis=1,inplace=True)
aplicaciones = pd.read_excel("entregable1.xlsx",sheet_name="data")

In [4]:
#Extraer fechas de inicio
fecha_inicio = aplicaciones.sort_values(by=["bloque","fecha_aplicacion"]).drop_duplicates(subset=["bloque"],keep='first')
dicc_fecha_inicio = {}
for index, row in fecha_inicio.iterrows():
    dicc_fecha_inicio[row["bloque"]] = row["fecha_aplicacion"]

In [35]:
aplicaciones_unicas = aplicaciones.sort_values(by=["bloque","fecha_aplicacion"]).drop_duplicates(subset=["bloque","homologación"],keep='first')[["fecha_aplicacion","bloque","homologación"]]

In [54]:
dicc_plan_aplicaciones = {}
for bloque in dicc_fecha_inicio.keys():

    fecha_inicial = dicc_fecha_inicio[bloque]
    calendario_temp = calendario_plan.copy()
    calendario_temp["bloque"]=bloque
    calendario_temp["fechas"]=calendario_temp["dias"].apply(lambda x: fecha_inicial +pd.Timedelta(days=x))
    ejecucion_temp = aplicaciones_unicas.query("bloque==@bloque").drop(["bloque"],axis=1)
    ejecucion_temp.columns = ["fecha_ejecucion","aplicacion"]
    calendario_temp = calendario_temp.merge(ejecucion_temp,how='left',on="aplicacion")
    calendario_temp["ejecucion-plan"]= calendario_temp["fecha_ejecucion"]-calendario_temp["fechas"]
    dicc_plan_aplicaciones[bloque] = calendario_temp.copy()

file_to_write = open("seguimiento_aplicaciones.pkl", "wb")
pickle.dump(dicc_plan_aplicaciones, file_to_write)

In [57]:
type(object_file)

dict

In [82]:
file = open("seguimiento_aplicaciones.pkl",'rb')
object_file = pickle.load(file)
file.close()
fechas_pendientes = pd.concat(object_file.values()).query('fecha_ejecucion in [@pd.NaT]')[["aplicacion","bloque","fechas"]]
fechas_pendientes["retraso"]=pd.Timestamp.today() - fechas_pendientes["fechas"]
fechas_pendientes["retraso"] = fechas_pendientes.retraso.dt.days
fechas_pendientes.query("retraso>0",inplace=True)
fechas_pendientes.sort_values(by="retraso",ascending=False,inplace=True)

In [83]:
fechas_pendientes

,aplicacion,bloque,fechas,retraso
8,Edafico 1,10619,2020-03-22,206
4,Protección 2,10120,2020-05-06,161
4,Protección 2,10419,2020-05-06,161
4,Protección 2,10220,2020-05-06,161
4,Protección 2,10519,2020-05-06,161
4,Protección 2,10320,2020-05-06,161
8,Edafico 1,10220,2020-05-31,136
8,Edafico 1,10519,2020-05-31,136
8,Edafico 1,10120,2020-05-31,136
8,Edafico 1,10419,2020-05-31,136


In [67]:
pd.Timestamp.today()

Timestamp('2020-10-14 08:29:18.430450')